In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings, sentence_transformer
from langchain.vectorstores import FAISS

DATA_PATH = r"C:\Users\mohni\OneDrive\Desktop\Llama2\data"
DB_FAISS_PATH = "vectorstores/db_faiss_index"

def create_vector_db():
    loader = DirectoryLoader(path=DATA_PATH, glob="*.pdf", loader_cls = PyPDFLoader)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap = 50)
    text = text_splitter.split_documents(data)

    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

    db = FAISS.from_documents(text, embeddings)
    db.save_local(DB_FAISS_PATH)

if __name__ == "__main__":
    create_vector_db()


In [3]:
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings, sentence_transformer
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
import streamlit as st

In [ ]:
# from langchain import PromptTemplate

DB_FAISS_PATH = r"C:\Users\mohni\OneDrive\Desktop\Krish\vectorstores\db_faiss_index"

prompt_template = """Use following pieces of information to answer the user's question.
    try to provide as much text as possible from "response". If you dont know the answer, please just say 
    "I dont know the answer". Don't try to make up an answer.
    
    Context : {context},
    Question : {question}
    
    Only return correct and helpful answer below and nothing else.
    Helful answer:

"""

def set_custom_prompt():
    prompt = PromptTemplate(template=prompt_template, input_variables= ["context", "question"])
    return prompt

def load_llm():
    llm = CTransformers(
      model = r"C:\Users\mohni\OneDrive\Desktop\Krish\llama-2-7b-chat.ggmlv3.q8_0.bin",
      model_type = "llama",
      max_new_tokens = 512,
      temperature = 0.5
  )
    return llm

def retrieval_qa_chain(llm,prompt, db):
    qa_chain = RetrievalQA.from_chain_type(llm = llm ,
                                         chain_type= "stuff",
                                         retriever = db.as_retriever(search_kwargs = {"k":2}),
                                         return_source_documents = True,
                                         chain_type_kwargs= {"prompt":prompt}
                                         )
    return qa_chain


def qa_chat():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    db = FAISS.load_local(DB_FAISS_PATH, embeddings)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)
    return qa

def final_result(query):
    qa_result = qa_chat()
    response = qa_result({"query":query})
    return response


In [ ]:
final_result("Ask your question here ?")

In [ ]:
# pip install ctransformers
import streamlit as st
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

## Function To get response from LLAma 2 model

def getLLamaresponse(input_text,no_words,blog_style):

    ### LLama2 model


    llm=CTransformers(model=r"C:\Users\mohni\OneDrive\Desktop\Krish\llama-2-7b-chat.ggmlv3.q8_0.bin",
                      model_type='llama',
                      config={'max_new_tokens':256,
                              'temperature':0.01})
    
    ## Prompt Template

    template="""
        Write a blog for {blog_style} job profile for a topic {input_text}
        within {no_words} words.
            """
    
    prompt=PromptTemplate(input_variables=["blog_style","input_text",'no_words'],
                          template=template)
    
    ## Generate the ressponse from the LLama 2 model
    response=llm(prompt.format(blog_style=blog_style,input_text=input_text,no_words=no_words))
    print(response)
    return response







In [ ]:
getLLamaresponse("what is facebook ?",10,"resercher")

In [ ]:

st.set_page_config(page_title="Generate Blogs",
                    page_icon='🤖',
                    layout='centered',
                    initial_sidebar_state='collapsed')

st.header("Generate Blogs 🤖")

input_text=st.text_input("Enter the Blog Topic")

## creating to more columns for additonal 2 fields

col1,col2=st.columns([5,5])

with col1:
    no_words=st.text_input('No of Words')
with col2:
    blog_style=st.selectbox('Writing the blog for',
                            ('Researchers','Data Scientist','Common People'),index=0)
    
submit=st.button("Generate")

## Final response
if submit:
    st.write(getLLamaresponse(input_text,no_words,blog_style))